<a href="https://colab.research.google.com/github/Sasch70/KI_basierte_Bildverarbeitung/blob/main/alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [ ]:
(ds_validate,ds_test,ds_train), ds_info = tfds.load(
    'fashion_mnist',
    split=['train[:10%]','train[10%:20%]','train[20%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
    data_dir="tensorflow_datasets"
)

In [ ]:
batch_size=16

# Konvertierung von uint8 Bild nach float32 Daten
def normalize_and_resize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  resized_img = tf.image.resize(image, (224,224))
  return resized_img, label
  #return tf.cast(resized_img, tf.float32) / 255., label

# Aufruf für alle Pixel
ds_train = ds_train.map(
    normalize_and_resize_img, num_parallel_calls=tf.data.AUTOTUNE)

# Daten in Memory laden
ds_train = ds_train.cache()
# Daten shufflen in Memory 
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)

# Batches herstellen mit jeweils 128 Bildern
ds_train = ds_train.batch(batch_size)
# Vorhalten von Daten für bessere Performanz, Autotune=dynamischer Puffer
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Normalisierung des validate Datensatzes
ds_validate = ds_validate.map(
    normalize_and_resize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_validate = ds_validate.batch(batch_size)
ds_validate = ds_validate.cache()
ds_validate = ds_validate.prefetch(tf.data.AUTOTUNE)

# Normalisierung des test Datensatzes
ds_test = ds_test.map(
    normalize_and_resize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
#l=list(ds_train.as_numpy_iterator())
#plt.imshow(l[0][0][5], cmap="gray")
#print("label:",l[0][1][5])


In [ ]:
from ctypes import resize


num_classes = 10
model = keras.Sequential(
    [
        tf.keras.layers.InputLayer([224,224,1]), # notwendig für summary()
        tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes)
    ]
)
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        11712     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 384)       1

In [ ]:
#input_shape = ds_info.features['image'].shape
#input_shape
#it = ds_train.as_numpy_iterator()
#el=it.next()

In [ ]:
# model kompilieren und Training starten
from gc import callbacks

max_epochs=5

if model: # auf None prüfen
    model.compile(
        #optimizer=keras.optimizers.Adam(0.005),
        optimizer=keras.optimizers.Adadelta(1.0),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    # Callbacks für TensorBoard-Aufzeichnung
    log_dir = "./logs/fit/fashion_mnist_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.fit(
        ds_train,
        epochs=max_epochs,
        validation_data=ds_validate,
        callbacks=[tensorboard_callback]
    )


Epoch 1/5
3000/3000 [==============================] - ETA: 0s - loss: 3.0370 - sparse_categorical_accuracy: 0.1013